## Analyzing Stack Overflow BigQuery Data

## Author
* Matthias Jiro Walther
* https://www.linkedin.com/in/jirowalther/

---

# Central Questions
1. Which tags have the highest scoring questions on Stack Overflow?

2. Are users commenting on their own posts or others' posts more?

3. What tags on posts are attracting the most comments?

---


# Dataset Analysis

## Detailed Overview



I'm using three Stack Overflow data tables:
- bigquery-public-data.stackoverflow.comments
-- 16.03 GB and 86,754,111 rows


- bigquery-public-data.stackoverflow.stackoverflow_posts
-- 29.36 GB and 31,017,889 rows

- bigquery-public-data.stackoverflow.users
-- 3.14 GB and 18,712,212 rows

In [12]:
from google.colab import auth
auth.authenticate_user()
project_id = "cs-145-project-1-474507"

In [13]:
# Initialize BiqQuery client
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

## Investigation

In [14]:
%%bigquery --project $project_id

# Title: Finding the most recent comment from each user

SELECT c.user_id, c.post_id, c.creation_date, c.score
FROM `bigquery-public-data.stackoverflow.comments` c
# Match the creation date with the latest date found in subquery
WHERE c.creation_date = (
  # subquery to find the latest comment date
  SELECT MAX(c2.creation_date)
  FROM `bigquery-public-data.stackoverflow.comments` c2
  WHERE c2.user_id = c.user_id
)
LIMIT 10;


Query is running:   0%|          |

Downloading:   0%|          |

,user_id,post_id,creation_date,score
0,448668,4868259,2012-10-25 12:46:24.240000+00:00,9
1,369065,8643124,2012-10-25 18:30:10.517000+00:00,12
2,1774864,13073261,2012-10-25 18:43:59.403000+00:00,6
3,637881,12285830,2012-10-25 19:59:50.813000+00:00,7
4,1757627,13072414,2012-10-26 23:06:14.093000+00:00,7
5,1778162,13093663,2012-10-27 00:22:40.683000+00:00,10
6,1783569,13126552,2012-10-29 21:44:22.923000+00:00,7
7,1282006,13128793,2012-10-30 01:06:12.583000+00:00,5
8,207028,11338583,2012-11-01 07:44:27.600000+00:00,7
9,1790814,13173752,2012-11-01 08:56:31.567000+00:00,10


In [15]:
%%bigquery --project $project_id

# Title: Finding the most recent POST of each user

SELECT p.id, p.owner_user_id, p.title, p.creation_date
FROM `bigquery-public-data.stackoverflow.stackoverflow_posts` p
# Match the creation date with the latest post date found in subquery
WHERE p.creation_date = (
  # Subquery to find latest post date
  SELECT MAX(p2.creation_date)
  FROM `bigquery-public-data.stackoverflow.stackoverflow_posts` p2
  WHERE p2.owner_user_id = p.owner_user_id
)
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,id,owner_user_id,title,creation_date
0,629073,4260,None,2009-03-10 06:03:18.427000+00:00
1,7777485,996781,None,2011-10-15 11:17:42.360000+00:00
2,3003853,362153,None,2010-06-09 07:34:33.353000+00:00
3,35221309,5887536,None,2016-02-05 10:07:00.520000+00:00
4,15047823,2103594,None,2013-02-24 01:51:15.017000+00:00
5,25044217,3841257,None,2014-07-30 18:26:08.403000+00:00
6,36654251,6174309,None,2016-04-15 18:18:18.090000+00:00
7,24888223,441063,None,2014-07-22 13:05:03.417000+00:00
8,36439660,6158399,None,2016-04-06 01:02:31.733000+00:00
9,17732041,2596919,None,2013-07-18 19:14:48.480000+00:00


In [16]:
%%bigquery --project $project_id

# Title: Finding the highest scoring post of users with over 100 up-votes

# CTE to find users who have over 100 up votes
WITH users_over_100_votes AS (
  SELECT id, reputation, up_votes, down_votes
  FROM `bigquery-public-data.stackoverflow.users`
  WHERE up_votes > 100
)

# Find the max post score for all users from CTE (limit 10)
SELECT p.owner_user_id, MAX(p.score) as max_score
FROM `bigquery-public-data.stackoverflow.stackoverflow_posts` p
JOIN users_over_100_votes u  # Join posts table to find posts made by users with over 100 up votes
ON p.owner_user_id = u.id
GROUP BY p.owner_user_id
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,owner_user_id,max_score
0,55520,49
1,43965,441
2,1462846,26
3,4026830,4
4,337251,41
5,222328,38
6,504357,363
7,86989,340
8,68701,16
9,232403,30


In [17]:
%%bigquery --project $project_id

# Title: Top 10 users with reputation over 1000 whose posts have the highest average scores

# Find users with reputation score over 1000
WITH reputable_users AS (
  SELECT id
  FROM `bigquery-public-data.stackoverflow.users`
  WHERE reputation > 1000
)

# Find users average score among those in reputable_users
SELECT p.owner_user_id, AVG(p.score) as avg_score
FROM `bigquery-public-data.stackoverflow.stackoverflow_posts` p
JOIN reputable_users r
ON p.owner_user_id = r.id
GROUP BY p.owner_user_id
# Show the highest average scores
ORDER BY avg_score DESC
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,owner_user_id,avg_score
0,13161,5005.0
1,8018,2778.0
2,233648,2566.0
3,188383,2334.0
4,268413,2076.0
5,441426,1671.0
6,543858,1664.0
7,774578,1597.0
8,581625,1519.0
9,595319,1418.0


In [18]:
%%bigquery --project $project_id

# Title: Finding the top 10 users with highest reputation ranks

SELECT id, display_name, reputation, RANK() OVER (ORDER BY reputation DESC) AS reputation_rank
FROM `bigquery-public-data.stackoverflow.users`
ORDER BY reputation DESC
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,id,display_name,reputation,reputation_rank
0,22656,Jon Skeet,1357603,1
1,1144035,Gordon Linoff,1211186,2
2,6309,VonC,1156679,3
3,157882,BalusC,1053483,4
4,29407,Darin Dimitrov,1003275,5
5,23354,Marc Gravell,989863,6
6,100297,Martijn Pieters,985470,7
7,157247,T.J. Crowder,980540,8
8,115145,CommonsWare,964739,9
9,893,Greg Hewgill,908125,10


In [19]:
%%bigquery --project $project_id

# Title: Top 10 highest-scoring posts among users top-scoring posts

SELECT owner_user_id, id, score
FROM  (
  # Inner query groups all posts by owner_user_id, orders their posts by score
  SELECT owner_user_id, id, score,
  # Assign rank to each post among a specific user
  RANK() OVER (PARTITION BY owner_user_id ORDER BY score DESC) AS score_rank
  FROM `bigquery-public-data.stackoverflow.stackoverflow_posts`
  WHERE owner_user_id IS NOT NULL AND owner_user_id >= 0    # don't include stackoverflow general community: owner_user_id = -1
)
# Get top scoring post
WHERE score_rank = 1
ORDER BY score DESC
LIMIT 10;


Query is running:   0%|          |

Downloading:   0%|          |

,owner_user_id,id,score
0,922184,11227902,20546
1,87234,11227809,14772
2,2057919,33617500,11941
3,14113,179147,11728
4,100297,29479702,10893
5,89904,927358,10584
6,62130,927386,10490
7,3093387,33741517,9368
8,8985,506004,8800
9,95592,2003515,8742


In [20]:
%%bigquery --project $project_id

# Title: Posts with scores higher than 100 with the lowest answer counts

SELECT id, owner_user_id, answer_count, body, score,
ROW_NUMBER() OVER (ORDER BY answer_count ASC) as answer_rank
FROM `bigquery-public-data.stackoverflow.stackoverflow_posts`
WHERE score > 100 and answer_count IS NOT NULL
ORDER BY answer_rank ASC
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,id,owner_user_id,answer_count,body,score,answer_rank
0,15051712,1321564,0,<p>I'm building a web application using Rails....,284,1
1,100358,6445,0,<blockquote> <p><strong>Possible Duplicate:</s...,112,2
2,23758875,440168,0,<p>There are a lot of questions about this pro...,441,3
3,30453656,1194864,0,<p>I want to store motion capture from kinect ...,114,4
4,5677271,127059,0,<blockquote> <p><strong>Possible Duplicate:</s...,133,5
5,6593592,444668,0,<p>Is there a way to get the android app versi...,309,6
6,8437307,162671,0,<p>I've been using <a href= http://www.jetbrai...,147,7
7,20674300,1134237,0,<p>I fall into a surprising issue. </p> <p>I l...,111,8
8,21428549,2412672,0,<p>I saw a loop which I've never seen before:<...,118,9
9,14791556,975959,0,<p>I wrote a tweak for Cydia it adds an icon t...,141,10


# Exploring Central Questions



## Question 1: Which tags have the highest scoring questions on Stack Overflow?

In [21]:
%%bigquery --project $project_id

# Find top 10 individual tags on Stack Overflow whose questions have the highest average score.
SELECT tag, AVG(p.score) AS avg_question_score
FROM `bigquery-public-data.stackoverflow.stackoverflow_posts` p
CROSS JOIN UNNEST(SPLIT(p.tags, '|')) AS tag        # Split tag strings into separate tags
WHERE p.tags IS NOT NULL AND p.post_type_id = 1     # No NULL and only from questions, not answers
GROUP BY tag
ORDER BY avg_question_score DESC                    # Order from highest to lowest average score
LIMIT 10;


Query is running:   0%|          |

Downloading:   0%|          |

,tag,avg_question_score
0,identity-operator,521.100000
1,multiline-comments,338.500000
2,hints-and-tips,332.500000
3,manual-retain-release,283.000000
4,c++-faq,269.797203
5,speculative,253.000000
6,tips-and-tricks,249.750000
7,equality-operator,234.523810
8,hidden-features,222.884615
9,hg-merge,213.000000


This query shows us the top 10 individual tags whose associated questions have the highest average scores, showing us which topics on Stack Overflow are being positively received by the community. The top three are 'identity-operator', 'multiline-comments', and 'hints-and-tips'. These seem to be relatively simple tags, possibly being associated with beginner-level coding questions. Because these topics are accessible and helpful to new programmers, they may be more positively engaged with and thus have higher question scores.

In [22]:
%%bigquery --project $project_id

# Finding the top users whose questions get the most positive reaction on average
# Score is based on average post score + comment score

WITH questions AS (
  # Fetch questions that have a known user who asked them
  SELECT id AS question_id, owner_user_id, score AS question_score
  FROM `bigquery-public-data.stackoverflow.stackoverflow_posts`
  WHERE post_type_id = 1 AND owner_user_id IS NOT NULL
),
comment_totals AS (
  # Fetch the sums of comment scores per post
  SELECT post_id AS question_id, SUM(score) AS comment_score_sum
  FROM `bigquery-public-data.stackoverflow.comments`
  GROUP BY post_id
),
joined AS (
  # Join questions to comment totals
  SELECT q.owner_user_id, q.question_id, q.question_score,
    IFNULL(c.comment_score_sum, 0) AS comment_score_sum   # Questions with no comments get a 0 (A.I. helped with this command)
  FROM questions q
  LEFT JOIN comment_totals c
  ON q.question_id = c.question_id
)

# For each asker of a question, compute average scores per question
SELECT owner_user_id, COUNT(*) AS n_questions, AVG(question_score) AS avg_question_score,
  AVG(comment_score_sum) AS avg_comment_score, AVG(question_score + comment_score_sum) AS avg_combined_score
FROM joined
GROUP BY owner_user_id
HAVING n_questions > 20
# Order users by their average combined score (question score average plus comment score average)
ORDER BY avg_combined_score DESC
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,owner_user_id,n_questions,avg_question_score,avg_comment_score,avg_combined_score
0,7473,33,312.545455,47.545455,360.090909
1,215945,29,201.068966,1.344828,202.413793
2,572670,25,118.440000,53.120000,171.560000
3,22656,39,135.589744,24.846154,160.435897
4,95592,67,130.686567,24.313433,155.000000
5,21709,44,149.136364,2.704545,151.840909
6,2041950,22,122.681818,22.863636,145.545455
7,3041,22,97.318182,44.863636,142.181818
8,18300,41,136.560976,3.292683,139.853659
9,117,24,117.791667,15.125000,132.916667


The results in this query show us the top ten Stack Overflow users whose questions are generating the most engagement, combining both the scores of their questions and their comments. We can see that these contributors are not only asking popular questions, but also facilitating conversation and exchanging knowledge with others.

## Question 2: Are users commenting on their own posts or others' posts more on Stack Overflow?

In [23]:
%%bigquery --project $project_id

# Finding how many comments are of users commenting on their own posts versus others.
SELECT
  # Identify when user id in comment matches user id of post
  CASE WHEN c.user_id = p.owner_user_id THEN 'self_comment'
  ELSE 'other_comment' END AS comment_type,
  COUNT(*) num_comments             # Find total number of comments that are self comments or comments on others
FROM `bigquery-public-data.stackoverflow.comments` c
JOIN `bigquery-public-data.stackoverflow.stackoverflow_posts` p
ON c.post_id = p.id                 # Joining based on post id
WHERE c.user_id IS NOT NULL AND p.owner_user_id IS NOT NULL
GROUP BY comment_type;

# other_comment refers to users' comments on others posts
# self_comment refers to users' comments on own posts

Query is running:   0%|          |

Downloading:   0%|          |

,comment_type,num_comments
0,other_comment,30393574
1,self_comment,13539871


We can see that of all the Stack Overflow comments, there were more than double the amount of comments on other users' posts versus commenting on one's own post. Users tend to comment on other users posts to answer their questions.

In [24]:
%%bigquery --project $project_id

# Counting self comments and comments on others posts, separated by posts that are questions vs. answers
SELECT
  CASE WHEN c.user_id = p.owner_user_id THEN 'self_comment'
  ELSE 'other_comment' END AS comment_type,
  p.post_type_id,
  COUNT(*) num_comments     # Find total number of comments that are self comments or comments on others
FROM `bigquery-public-data.stackoverflow.comments` c
JOIN `bigquery-public-data.stackoverflow.stackoverflow_posts` p
ON c.post_id = p.id         # Joining based on post id
WHERE c.user_id IS NOT NULL AND p.owner_user_id IS NOT NULL AND p.post_type_id IN (1, 2)
# Group by the type of comment and the type of post (i.e. self comment vs. other comment, question vs. answer)
GROUP BY comment_type, p.post_type_id
ORDER BY p.post_type_id, comment_type;

Query is running:   0%|          |

Downloading:   0%|          |

,comment_type,post_type_id,num_comments
0,other_comment,1,13853874
1,self_comment,1,6428441
2,other_comment,2,16536298
3,self_comment,2,7110168


After differentiating between post types, specifically posts that are questions or answers, we can see that users are more likely to comment on other users' posts rather than their own. However, we can see that self-commenting isn't rare. In fact, we can see that around 1 in 3 comments is from the original poster, suggesting that oftentimes there are follow-ups or responses to feedback from others.

## Question 3: What tags on Stack Overflow posts are attracting the most comments?

In [25]:
%%bigquery --project $project_id

# Versus using comment_count in posts table, this query counts what's in the
# comments table which is slower but more precise

# CTE to count how many comments each post has
WITH per_post_comments AS (
  SELECT post_id, COUNT(*) as num_comments
  FROM `bigquery-public-data.stackoverflow.comments`
  GROUP BY post_id
)

# Find which tag combintations have the most total comments
SELECT p.tags, SUM(pc.num_comments) AS total_comments, COUNT(*) AS num_questions
FROM `bigquery-public-data.stackoverflow.stackoverflow_posts` p
JOIN per_post_comments pc          # Join CTE using post id's to get comment counts
ON p.id = pc.post_id
WHERE p.post_type_id = 1 AND p.tags IS NOT NULL
GROUP BY p.tags
ORDER BY total_comments DESC
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,tags,total_comments,num_questions
0,java,171745,48638
1,php,165689,46793
2,javascript,152230,44322
3,javascript|jquery,151590,44201
4,android,130846,45597
5,c++,124334,29832
6,php|mysql,119045,30469
7,c#,113191,31602
8,python,106400,32401
9,html|css,101451,31261


Based on this query, we can see that the posts with the tag 'java' in the question have the most comments. Following 'java', the tags 'php' and 'javascript' are the second and third most commented on posts.

In [26]:
%%bigquery --project $project_id

# Use the comment_count column to identify tags in posts that garner the most comments
SELECT p.tags, SUM(p.comment_count) AS total_comments
FROM `bigquery-public-data.stackoverflow.stackoverflow_posts` p
WHERE p.tags IS NOT NULL and p.post_type_id = 1
GROUP BY p.tags
# Order by the total comments under posts with a specific tag, showing the highest first
ORDER BY total_comments DESC
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,tags,total_comments
0,java,184183
1,php,179985
2,javascript,161204
3,javascript|jquery,160422
4,android,146384
5,c++,132116
6,php|mysql,130185
7,c#,121795
8,python,113356
9,html|css,107435


This query aims to answer the same question as the previous query, however, it does not utilize the comment table and instead uses the total from the posts table. I believe that the previous query will result in more accurate data as it is actually counting every comment in the comments table. It's reassuring to see that the top 10 tags remain the same across both queries.

# Findings


**Top Tags by Average Question Score:**

* The top 10 tags with the highest average question scores reveal which Stack Overflow topics are most positively received.

* The top three tags are ‘identity-operator,’ ‘multiline-comments,’ and ‘hints-and-tips.’

* These tags are likely tied to beginner-level or accessible topics, which may explain their higher engagement and positive reception.

**Top Users by Engagement:**
* The top 10 users were identified based on the combined scores of their questions and comments.

* These users not only ask popular questions but also drive conversations and knowledge sharing within the community.

**Commenting Behavior:**

* There are more than twice as many comments on other users’ posts compared to comments on one’s own posts.

* Users typically comment to help answer questions or contribute to discussions on others’ posts.

* However, self-commenting is fairly common — about 1 in 3 comments is made by the original poster.

* This suggests that users often follow up, clarify, or respond to feedback on their own posts.

**Query Comparison and Data Accuracy:**

* A secondary query was run using the posts table instead of the comments table to measure engagement.

* The previous query (using the comments table) is considered more accurate, as it counts every individual comment.

* It’s reassuring that the top 10 tags remained consistent across both methods, reinforcing the reliability of the findings.